In [1]:
import pandas as pd

# Step 1: Load dataset
df = pd.read_csv("EGA_data.csv",sep=';',  parse_dates=['OrderDate', 'ShipDate']) ##allow to read dates
df = df.sort_values('OrderDate').reset_index(drop=True)

In [2]:
#Compute Z-score for Sales and Profit


for col in ['Sales', 'Profit', 'Discounts']:
    mean = df[col].mean()
    std = df[col].std()
    df[f'{col}_zscore'] = (df[col] - mean) / std

    # Step 2: Replace outliers with mean if |z| > 3
    df.loc[df[f'{col}_zscore'].abs() > 3, col] = mean

In [3]:
!pip install sdv>=1.20.0

from sdv.single_table import CTGANSynthesizer
from sdv.metadata import SingleTableMetadata # Import the metadata class
import pandas as pd

df = pd.read_csv("EGA_data.csv", sep=';', parse_dates=['OrderDate','ShipDate'])
df = df.drop(columns=['OrderID'])  # drop ID col

# Instantiate the metadata object
metadata = SingleTableMetadata()

# Detect metadata from the DataFrame
metadata.detect_from_dataframe(data=df)

# Instantiate the synthesizer with the generated metadata
synth = CTGANSynthesizer(metadata=metadata, epochs=500, verbose=True)
synth.fit(df)
fake = synth.sample(1000)

/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:162: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:128: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
Gen. (-0.17) | Discrim. (-0.05): 100%|██████████| 500/500 [13:20<00:00,  1.60s/it]


In [4]:
print(fake.head())
print(fake.dtypes)

   OrderDate   ShipDate        ShipMode         State   Category   Sales  \
0 2020-02-24 2020-10-05    Second Class          Iowa  Furniture   36.58   
1 2018-09-09 2018-09-30    Second Class       Georgia  Furniture  131.68   
2 2020-12-13 2021-03-17    Second Class       Montana  Furniture  261.96   
3 2018-08-24 2018-09-27  Standard Class      Virginia  Furniture  131.24   
4 2021-09-22 2021-09-21     First Class  North Dakota  Furniture  130.20   

   Profit  Discounts  
0   12.23       0.73  
1   52.90       0.00  
2   -2.55       0.00  
3  -41.27       0.68  
4   15.09       0.00  
OrderDate    datetime64[ns]
ShipDate     datetime64[ns]
ShipMode             object
State                object
Category             object
Sales               float64
Profit              float64
Discounts           float64
dtype: object


In [5]:
#Date format conversion
fake['OrderDate'] = pd.to_datetime(fake['OrderDate'], errors='coerce')
fake['ShipDate'] = pd.to_datetime(fake['ShipDate'], errors='coerce')

In [6]:
#create orderDate & ShipDate
import numpy as np

fake['OrderDate'] = pd.to_datetime(np.random.choice(pd.date_range('2022-01-01', '2023-12-31'), size=len(fake)))
fake['ShipDate'] = fake['OrderDate'] + pd.to_timedelta(np.random.randint(1, 7, size=len(fake)), unit='D')

In [ ]:
#formatting and downloading file
import pandas as pd

fake.to_csv('EGA_synthetic_2022_2023.csv', index=False)

df = pd.read_csv('EGA_synthetic_2022_2023.csv', parse_dates=['OrderDate', 'ShipDate'])
original_df = pd.read_csv("EGA_data.csv", sep=';', parse_dates=['OrderDate', 'ShipDate'])

# Filter and sort
df = df[df['Category'] == 'Furniture']
df = df.sort_values(by='OrderDate')

# Step 4: Save the updated data to a new CSV
df.to_csv('EGA_synthetic_sorted.csv', index=False)

# Drop duplicate or conflicting columns like 'OrderID' if needed
original_df = original_df.drop(columns=['OrderID'], errors='ignore')

# Combine both DataFrames
# Corrected the variable name from EGA_synthetic_sorted to df
combined_df = pd.concat([original_df, df], ignore_index=True)

# Optional: sort by date if needed
combined_df = combined_df.sort_values(by='OrderDate').reset_index(drop=True)

# Save to new CSV
combined_df.to_csv("EGA_combined_real_synthetic.csv", index=False)

from google.colab import files
files.download('EGA_combined_real_synthetic.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>